In [1]:
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch import nn
from torch.optim import SGD
from sklearn.metrics import accuracy_score
from os.path import join
import torch
import numpy as np
from PIL import Image
from os import path

class ScenesDataset(Dataset):
    
    def __init__(self,base_path,txt_list,transform=None):
        self.base_path=base_path
        self.images = np.loadtxt(txt_list,dtype=str,delimiter=',')
        self.transform = transform
        
    def __getitem__(self, index):
        f,c = self.images[index]
        im = Image.open(path.join(self.base_path, f))
        if self.transform is not None:
            im = self.transform(im)
        
        label = int(c)
        
        return {'image' : im, 'label':label}
    
    def __len__(self):
        return len(self.images)

In [2]:
#resize normale
transform = transforms.Compose([transforms.Resize(16),transforms.ToTensor()])

dataset = ScenesDataset('DITS-full\DITS-full\DITS-detection\class\image','DITS-full\DITS-full\DITS-detection\class\classes.txt',transform=transform)
sample = dataset[10]
print(sample['image'].shape)
print(sample['label'])


datatrain = ScenesDataset('DITS-full\DITS-full\DITS-detection\detection_test\day','DITS-full\DITS-full\DITS-detection\detection_test\day\classes.txt',transform=transform)

torch.Size([3, 16, 16])
0


for i in range(len((dataset))):
    sample = dataset[i]
    print(sample['image'].shape)
    print(sample['label'])

print(sample['image'].shape[1])
print(sample['image'].shape[2])
print(sample['image'].sum(2).sum(1))
print(sample['image'])

In [3]:
dataset = ScenesDataset('DITS-full\DITS-full\DITS-detection\class\image','DITS-full\DITS-full\DITS-detection\class\classes.txt',transform=transform)

m = np.zeros(3)
s = np.zeros(3)
peso = 0
for sample in dataset:
    m+=sample['image'].sum(1).sum(1).numpy()
    peso+=sample['image'].shape[1]*sample['image'].shape[2]
    
m=m/(len(dataset)*peso)

for sample in dataset:
    s+=((sample['image']-torch.Tensor(m).view(3,1,1))**2).sum(1).sum(1).numpy()
    
s=np.sqrt(s/(len(dataset)*peso))

print("Medie",m)
print("Dev. Std.",s)

Medie [0.00035388 0.00032966 0.00032453]
Dev. Std. [0.01475199 0.01385172 0.01393348]


In [4]:
class AverageValueMeter():
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.sum = 0
        self.num = 0
        
    def add(self, value, num):
        self.sum += value*num
        self.num += num
        
    def value(self):
        try :
            return self.sum/ self.num
        except :
            return None

In [5]:
def train_classifier(model, train_loader, test_loader, exp_name='experiment' ,
                     lr=0.01, epochs=10, momentum=0.99, logdir='logs'):
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr, momentum=momentum)
    #meters
    loss_meter = AverageValueMeter()
    acc_meter = AverageValueMeter()
    #writer
    writer = SummaryWriter(join(logdir, exp_name))
    #device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    #definiamo un dizionario contenente i Loader di training e test
    loader = {
        'train' : train_loader,
        'test' : test_loader
    }
    #iniziaLizziamo iL global step
    global_step = 0
    for e in range(epochs):
        #iteriamo tra due modalità: train e test
        for mode in ['train' , 'test']:
            loss_meter.reset(); acc_meter.reset()
            model.train() if mode == 'train' else model.eval()
            with torch.set_grad_enabled(mode=='train'): #abiLitiamo i gradienti SOLO in training
                for i, batch in enumerate(loader[mode]):
                    x=batch[0].to(device) #"portiamoLi sul device corretto"
                    y=batch[1].to(device)
                    output = model(x)
                    
                    #aggiorniamo iL gLobaL_step
                    #conterrà iL numero di campioni visti durante iL training
                    n = x.shape[0] #numero di elementi nel batch
                    global_step += n
                    l = criterion(output, y)
                    
                    if mode=='train' :
                        l.backward()
                        optimizer.step( )
                        optimizer.zero_grad ( )
                    
                    acc = accuracy_score(y.to('cpu '),output.to(' cpu').max(1)[1])
                    loss_meter.add(l.item(), n)
                    acc_meter.add(acc,n)
                    
                    #Loggiamo i risultati iterazione per iterazione SOLO durante iL training
                    if mode=='train' :
                        writer.add_scalar( ' loss/train ' ,loss_meter.value(), global_step=global_step)
                        writer.add_scalar( 'accuracy/train' ,acc_meter.value(), global_step=global_step)            
                #una voLta finita L 'epoca (sia nel caso di training che test, Loggiamo Le stime finali)
                writer.add_scalar( 'loss/' + mode, loss_meter.value(), global_step=global_step)
                writer.add_scalar( 'accuracy/' + mode, acc_meter.value(), global_step=global_step)
        #conserviamo i pesi del model Lo aLLa fine di un ciclo di training e test
        torch. save(model.state_dict(), '%s-%d.pth'%(exp_name,e+1))
    return model

In [6]:
class LeNetColor(nn.Module):
    def __init__ (self):
        super(LeNetColor, self).__init__()
        #ridefiniamo iL modeLLo utilizzando i moduli sequentiaL.
        #ne definiamo due: un "feature extractor", che estrae Le feature maps
        #e un "classificatore" che implementa i LiveLLy FC
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 18, 5), #Input: 3 x 32 x 32. Ouput: 18 x 28 x 28
            nn.MaxPool2d(2), #Input: 18 x 28 x 28. Output: 18 x 14 x 14
            nn.ReLU(),
            nn.Conv2d(18, 28, 5), #lnput 18 x 14 x 14. Output: 28 x le x le
            nn.MaxPool2d(2), #Input 28 x le x le. Output: 28 x 5 x 5
            nn.ReLU()
        )    
            
        self.classifier = nn.Sequential(
            nn.Linear(700, 360), #rnput: 28 * 5 * 5
            nn.ReLU(),
            nn.Linear(360, 252),
            nn.ReLU(),
            nn.Linear(252, 100)
        )
        
    def forward(self,x):
        #AppLichiamo Le diverse trasformazioni in cascata
        x = self.feature_extractor(x)
        x = self.classifier(x.view(x.shape[0],-1))
        return x

In [7]:
dataset_loader = DataLoader(dataset, batch_size=1024, num_workers=2, shuffle=True)
#cifar100_test_loader = DataLoader(cifar100_test, batch_size=1024, num_workers=2)
datatrain_loader = DataLoader(datatrain, batch_size=1024, num_workers=2, shuffle=True)
#cifar100_test_loader = DataLoader(cifar100_test, batch_size=1024, num_workers=2)

In [ ]:
lenet_dataset = LeNetColor()
lenet_dataset = train_classifier(lenet_dataset, dataset_loader, datatrain_loader, \
                                 'lenet_dataset', epochs=1)